<a href="https://colab.research.google.com/github/jerome-keli/Deep_Learning/blob/main/TensorFlow_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Neural Network for MNIST Classification

We'll apply all the knowledge from the lectures in this section to write a deep neural network. The problem we've chosen is referred to as the "Hello World" of deep learning because for most students it is the first deep learning algorithm they see.

The dataset is called MNIST and refers to handwritten digit recognition. You can find more about it on Yann LeCun's website (Director of AI Research, Facebook). He is one of the pioneers of what we've been talking about and of more complex approaches that are widely used today, such as covolutional neural networks (CNNs).

The dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image).

The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes.

Our goal would be to build a neural network with 2 hidden layers.

## Import the relevant packages

In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

## Data

That's where we load and preprocess our data.

In [2]:
mnist_dataset,mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.8ONEHR_3.0.1/mnist-train.tfrecord*...:   0%|          | 0…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.8ONEHR_3.0.1/mnist-test.tfrecord*...:   0%|          | 0/…

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [3]:
mnist_train,mnist_test = mnist_dataset['train'], mnist_dataset['test']

mnist_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
mnist_validation_samples = tf.cast(mnist_validation_samples, tf.int64)

mnist_test_samples = 0.1 * mnist_info.splits['test'].num_examples
mnist_test_samples = tf.cast(mnist_test_samples, tf.int64)

In [4]:
def scale(image,label):
  image = tf.cast(image, tf.float32)
  image /=255.
  return image,label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [5]:
BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(mnist_validation_samples)
train_data = shuffled_train_and_validation_data.skip(mnist_validation_samples)

BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(mnist_validation_samples)
test_data = test_data.batch(mnist_test_samples)

## Model

Outline the model

In [6]:
input_data = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(input_shape=(28,28,1)),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(output_size, activation='softmax')

])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Choose the optimizer and the loss function

In [7]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy']) #use scc because it does one hot encoding for us

Training

In [8]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data = validation_data, verbose=2)

Epoch 1/5
540/540 - 11s - 21ms/step - accuracy: 0.9196 - loss: 0.2763 - val_accuracy: 0.9580 - val_loss: 0.1310
Epoch 2/5
540/540 - 6s - 11ms/step - accuracy: 0.9686 - loss: 0.1042 - val_accuracy: 0.9778 - val_loss: 0.0745
Epoch 3/5
540/540 - 14s - 26ms/step - accuracy: 0.9782 - loss: 0.0707 - val_accuracy: 0.9760 - val_loss: 0.0771
Epoch 4/5
540/540 - 11s - 20ms/step - accuracy: 0.9841 - loss: 0.0506 - val_accuracy: 0.9802 - val_loss: 0.0658
Epoch 5/5
540/540 - 6s - 12ms/step - accuracy: 0.9877 - loss: 0.0388 - val_accuracy: 0.9875 - val_loss: 0.0424


Test the model

In [10]:
test_loss,test_accuracy = model.evaluate(test_data)
print('Test loss: {0:.2f}, Test accuracy: {1:.2f}%'.format(test_loss,test_accuracy*100.))

10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 238ms/step - accuracy: 0.9771 - loss: 0.0721
Test loss: 0.07, Test accuracy: 97.87%
